In [1]:
%load_ext autoreload
%autoreload 2
from scripts.agents import Tool, InternalTool, CustomAgent, CustomGraph, InternalToolInput
from dotenv import load_dotenv
load_dotenv()



c:\Users\cgwed\anaconda3\envs\llm_hub\Lib\site-packages\pydantic\_internal\_generate_schema.py:547: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [2]:
from langchain_mistralai import ChatMistralAI
from langchain_aws import ChatBedrock
# Define the Bedrock client using LangChain
#llm = ChatMistralAI(
#    model="mistral-small-latest"
#  )  # Replace with the specific Bedrock model you want to use

llm = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0",
    region_name='us-east-1',
    model_kwargs = {
        'temperature': 0
    }
)


from langchain.agents import AgentType
from langgraph.graph import START, END

# Define tools for each operation
def add(a, b):
    return a + b

def subtract(a, b):
    return a - b

def multiply(a, b):
    return a * b



add_tool = InternalTool(
        name="Addition",
        description="Add two numbers together",
        function=add
    )

subtract_tool = InternalTool(
        name="SubtractTool",
        description="Subtract two numbers",
        function=subtract
    )

multiply_tool = InternalTool(
        name="ExampleTool",
        description="An example tool for testing",
        function=multiply
    )

#start_agent = CustomAgent("StartAgent", llm, [start_tool], 

# Create CustomAgent instances
add_agent = CustomAgent("AddAgent", llm, [add_tool], AgentType.ZERO_SHOT_REACT_DESCRIPTION)
subtract_agent = CustomAgent("SubtractAgent", llm, [subtract_tool], AgentType.ZERO_SHOT_REACT_DESCRIPTION)
multiply_agent = CustomAgent("MultiplyAgent", llm, [multiply_tool], AgentType.ZERO_SHOT_REACT_DESCRIPTION)

def make_available_nodes(nodes):
    available_nodes = {node.name: node for node in nodes}
    #available_nodes['START'] = START
    #available_nodes['END'] = END
    return available_nodes
available_nodes = make_available_nodes([add_agent, subtract_agent, multiply_agent])
start_agent = 'AddAgent'
end_agents = ['AddAgent', 'SubtractAgent', 'MultiplyAgent']
outline = {'START': ['AddAgent'],
           'AddAgent': ['SubtractAgent', 'MultiplyAgent', 'END'],
           'SubtractAgent': ['AddAgent', 'MultiplyAgent', 'END'],
           'MultiplyAgent': ['SubtractAgent', 'AddAgent', 'END']}


In [3]:
graph = CustomGraph(outline=outline)
graph.create_graph(available_nodes=available_nodes)

2025-01-28 20:10:44,368 - scripts.agents - DEBUG - Adding Nodes
2025-01-28 20:10:44,369 - scripts.agents - DEBUG - Getting node AddAgent
2025-01-28 20:10:44,369 - scripts.agents - DEBUG - Adding node AddAgent to workflow
2025-01-28 20:10:44,371 - scripts.agents - DEBUG - Getting node SubtractAgent
2025-01-28 20:10:44,372 - scripts.agents - DEBUG - Adding node SubtractAgent to workflow
2025-01-28 20:10:44,373 - scripts.agents - DEBUG - Getting node MultiplyAgent
2025-01-28 20:10:44,374 - scripts.agents - DEBUG - Adding node MultiplyAgent to workflow
2025-01-28 20:10:44,375 - scripts.agents - DEBUG - Start Node: AddAgent
2025-01-28 20:10:44,375 - scripts.agents - DEBUG - Adding Edges
2025-01-28 20:10:44,376 - scripts.agents - DEBUG - Adding edge from START to AddAgent
2025-01-28 20:10:44,377 - scripts.agents - DEBUG - Processing edges for AddAgent
2025-01-28 20:10:44,378 - scripts.agents - DEBUG - Adding edge connecting AddAgent to other nodes
2025-01-28 20:10:44,379 - scripts.agents - D

In [4]:
state = {
    "messages": [{"agent": "START", "content": "5 10 20"}],  # User input
    "current_agent": "START",
    "available_nodes": ["AddAgent", "SubtractAgent"],
    "node_types": {"AddAgent": "agent", "SubtractAgent": "agent", "MultiplyAgent": "agent", "END": "end"},
    "finished": False,
}

#from langchain_core.messages import HumanMessage
#result = graph.graph.invoke(HumanMessage("5 10 20"))
# Execute the workflow
"""
current_node = "START"
while current_node != "END":
    print(f"Current Node: {current_node}")
    state = graph.graph.invoke(current_node, state)
    print(f"Messages: {state['messages']}")
    current_node = state["current_agent"]

print("Workflow completed.")
"""

'\ncurrent_node = "START"\nwhile current_node != "END":\n    print(f"Current Node: {current_node}")\n    state = graph.graph.invoke(current_node, state)\n    print(f"Messages: {state[\'messages\']}")\n    current_node = state["current_agent"]\n\nprint("Workflow completed.")\n'

In [10]:

from langchain_core.messages import HumanMessage
from scripts.agents import NodeState
message = 'Solve this problem: (2+3)-4*8'
initial_state = NodeState(
    messages = [HumanMessage(content=message, role="user")],
    current_node="AddAgent",
    available_nodes=["MultiplyAgent", "SubtractAgent"],
    finished=False
)
#result = graph.graph.invoke(initial_state)


In [11]:
graph.graph.get_graph()

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=<class 'langchain_core.utils.pydantic.LangGraphInput'>, metadata=None), 'AddAgent': Node(id='AddAgent', name='AddAgent', data=AddAgent(tags=None, recurse=True, func_accepts_config=False, func_accepts={'writer': False, 'store': False}), metadata=None), 'SubtractAgent': Node(id='SubtractAgent', name='SubtractAgent', data=SubtractAgent(tags=None, recurse=True, func_accepts_config=False, func_accepts={'writer': False, 'store': False}), metadata=None), 'MultiplyAgent': Node(id='MultiplyAgent', name='MultiplyAgent', data=MultiplyAgent(tags=None, recurse=True, func_accepts_config=False, func_accepts={'writer': False, 'store': False}), metadata=None), '__end__': Node(id='__end__', name='__end__', data=<class 'langchain_core.utils.pydantic.LangGraphOutput'>, metadata=None)}, edges=[Edge(source='__start__', target='AddAgent', data=None, conditional=False), Edge(source='AddAgent', target='SubtractAgent', data=None, conditional=

In [12]:
initial_state

{'messages': [HumanMessage(content='Solve this problem: (2+3)-4*8', additional_kwargs={}, response_metadata={}, role='user')],
 'current_node': 'AddAgent',
 'available_nodes': ['MultiplyAgent', 'SubtractAgent'],
 'finished': False}

In [13]:
initial_state['messages'][-1].content

'Solve this problem: (2+3)-4*8'

In [15]:
result = graph.graph.invoke({"messages": [HumanMessage(content=message, role="user")]})

2025-01-28 20:44:03,855 - scripts.agents - DEBUG - Processing for Node MultiplyAgent
2025-01-28 20:44:03,857 - scripts.agents - DEBUG - Input State: {'messages': [HumanMessage(content='Solve this problem: (2+3)-4*8', additional_kwargs={}, response_metadata={}, role='user')]}
2025-01-28 20:44:03,858 - scripts.agents - DEBUG - Invoking MultiplyAgent
2025-01-28 20:44:05,750 - scripts.agents - DEBUG - Reuslt of MultiplyAgent invocation: {'messages': [HumanMessage(content='Solve this problem: (2+3)-4*8', additional_kwargs={}, response_metadata={}, id='5fc0167f-af22-4e06-a0a7-e9f02a0ba22f', role='user'), AIMessage(content='To solve this problem, we need to follow the order of operations (PEMDAS):\n\n1. Evaluate the expressions inside the parentheses: 2+3 = 5\n2. Multiply 4 and 8: 4*8 = 32\n3. Subtract 32 from 5: 5 - 32 = -27\n\nSo the final answer is -27.', additional_kwargs={'usage': {'prompt_tokens': 24, 'completion_tokens': 79, 'total_tokens': 103}, 'stop_reason': 'stop', 'model_id': 'met

ValueError: Message dict must contain 'role' and 'content' keys, got {'messages': [HumanMessage(content='Solve this problem: (2+3)-4*8', additional_kwargs={}, response_metadata={}, id='5fc0167f-af22-4e06-a0a7-e9f02a0ba22f', role='user'), AIMessage(content='To solve this problem, we need to follow the order of operations (PEMDAS):\n\n1. Evaluate the expressions inside the parentheses: 2+3 = 5\n2. Multiply 4 and 8: 4*8 = 32\n3. Subtract 32 from 5: 5 - 32 = -27\n\nSo the final answer is -27.', additional_kwargs={'usage': {'prompt_tokens': 24, 'completion_tokens': 79, 'total_tokens': 103}, 'stop_reason': 'stop', 'model_id': 'meta.llama3-8b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 24, 'completion_tokens': 79, 'total_tokens': 103}, 'stop_reason': 'stop', 'model_id': 'meta.llama3-8b-instruct-v1:0'}, id='run-5a55803a-82cb-455e-aa5b-1a370662f7c7-0', usage_metadata={'input_tokens': 24, 'output_tokens': 79, 'total_tokens': 103})]}
For troubleshooting, visit: https://python.l

In [32]:
add_agent.invoke()

TypeError: CustomAgent.invoke() missing 2 required positional arguments: 'state' and 'query'

In [45]:
import inspect
print(inspect.getsource(add_agent.invoke))

    def invoke(self, state):
        """
        Run the agent with the user prompt

        Returns:
            _type_: _description_
        """
        logging.debug(f'Invoking {self.name}')
        response = self.agent.invoke(state)
        return Command(
            update={
                "messages": [
                    HumanMessage(content=response["messages"][-1].content, name=self.name)
                ]
            }
        )
        #state["messages"].append({"node": self.name, "content": response})

        return state

